<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [26]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import os
import numpy as np
import pandas as pd
import datetime
import time
import math

import numpy as np
import pandas as pd

import multiprocessing
from multiprocessing import Pool

from multiprocessing import Process
from multiprocessing import Array
from multiprocessing import Manager
from multiprocessing import queues
from multiprocessing import RLock, Lock, Event, Condition, Semaphore

import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

---

In [34]:
def info(title):
    print(title)
    print('module name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())
    print('')

def f(name):
    print('function f')
    print('module name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())
    print('')
    print('hello', name)

info('main line')

p = Process(target=f, args=('bob',))
p.start()
p.join()

main line
module name: __main__
parent process: 23267
process id: 1982

function f
module name: __main__
parent process: 1982
process id: 2293

hello bob


In [35]:
lis = []

def foo(i):
    lis.append(i)
    
    print("This is Process: {0}, and lis is: {1}, and lis.address is: {2}".format(i, lis, id(lis)))


for i in range(5):
    p = Process(target = foo, args=(i, )) # i,
    p.start()
    
print("The end of list_1:", lis)

This is Process: 0, and lis is: [0], and lis.address is: 140646338525456
This is Process: 1, and lis is: [1], and lis.address is: 140646338525456
The end of list_1: []
This is Process: 2, and lis is: [2], and lis.address is: 140646338525456
This is Process: 3, and lis is: [3], and lis.address is: 140646338525456
This is Process: 4, and lis is: [4], and lis.address is: 140646338525456


In [36]:
def func(i,temp):
    temp[0] += 100
    print("进程 {0}, 修改数组第一个元素后-----> {1}".format(i, temp[0]))

    
# 'c': ctypes.c_char, 'u': ctypes.c_wchar,
# 'b': ctypes.c_byte, 'B': ctypes.c_ubyte,
# 'h': ctypes.c_short, 'H': ctypes.c_ushort,
# 'i': ctypes.c_int, 'I': ctypes.c_uint,
# 'l': ctypes.c_long, 'L': ctypes.c_ulong,
# 'f': ctypes.c_float, 'd': ctypes.c_double
temp = Array('i', [1, 2, 3, 4])
for i in range(10):
    p = Process(target = func, args=(i, temp))
    p.start()

进程 0, 修改数组第一个元素后-----> 101
进程 1, 修改数组第一个元素后-----> 201
进程 2, 修改数组第一个元素后-----> 301
进程 3, 修改数组第一个元素后-----> 401
进程 4, 修改数组第一个元素后-----> 501
进程 5, 修改数组第一个元素后-----> 601
进程 6, 修改数组第一个元素后-----> 701
进程 7, 修改数组第一个元素后-----> 801
进程 8, 修改数组第一个元素后-----> 901
进程 9, 修改数组第一个元素后-----> 1001


In [37]:
def func(i, dic):
    dic["num"] = 100+i
    print(dic.items())

if __name__ == '__main__':
    dic = Manager().dict()
    for i in range(10):
        p = Process(target = func, args = (i, dic))
        p.start()
        p.join()

[('num', 100)]
[('num', 101)]
[('num', 102)]
[('num', 103)]
[('num', 104)]
[('num', 105)]
[('num', 106)]
[('num', 107)]
[('num', 108)]
[('num', 109)]


In [38]:
def func(i, q):
    ret = q.get()
    print("进程 {0} 从队列里获取了一个 {1}，然后又向队列里放入了一个 {2}".format(i, ret, i))
    q.put(i)

    
lis = queues.Queue(20, ctx=multiprocessing)
lis.put(0)
for i in range(10):
    p = Process(target=func, args=(i, lis,))
    p.start()

进程 0 从队列里获取了一个 0，然后又向队列里放入了一个 0
进程 1 从队列里获取了一个 0，然后又向队列里放入了一个 1
进程 2 从队列里获取了一个 1，然后又向队列里放入了一个 2
进程 3 从队列里获取了一个 2，然后又向队列里放入了一个 3
进程 4 从队列里获取了一个 3，然后又向队列里放入了一个 4
进程 5 从队列里获取了一个 4，然后又向队列里放入了一个 5
进程 6 从队列里获取了一个 5，然后又向队列里放入了一个 6
进程 7 从队列里获取了一个 6，然后又向队列里放入了一个 7
进程 8 从队列里获取了一个 7，然后又向队列里放入了一个 8
进程 9 从队列里获取了一个 8，然后又向队列里放入了一个 9


In [39]:
def func(i,lis,lc):
    lc.acquire()
    lis[0] = lis[0] - 1
    time.sleep(1)
    print('say hi', lis[0])
    lc.release()


array = Array('i', 1)
array[0] = 10
lock = RLock()
for i in range(10):
    p = Process(target=func, args=(i, array, lock))
    p.start()

say hi 9
say hi 8
say hi 7
say hi 6
say hi 5
say hi 4
say hi 3
say hi 2
say hi 1
say hi 0


In [ ]:
from multiprocessing import Pool
import time

def func(args):
    time.sleep(1)
    print("正在执行进程 ", args)


p = Pool(4)     # 创建一个包含4个进程的进程池
for i in range(30):
    p.apply_async(func=func, args=(i,))

p.close()           # 等子进程执行完毕后关闭进程池
# time.sleep(2)
# p.terminate()     # 立刻关闭进程池
p.join()